In [4]:
import os
os.environ["SPARK_HOME"] = "/home/anyes/spark"
os.environ["JAVA_HOME"] = "/usr"

# Importation des bibliothéques

In [16]:
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# for dataframe and udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Lancer Spark

In [5]:


# initialise environment variables for spark
findspark.init()

# Start spark session
# --------------------------
def start_spark():
  local = "local[*]"
  appName = "PLDAC"

  gf = "graphframes:graphframes:0.8.3-spark3.5-s_2.12"

  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.jars.packages", gf)

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # Adjust the query execution environment to the size of the cluster (4 cores)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session started, its id is ", sc.applicationId)
  return spark
spark = start_spark()

24/02/23 15:42:11 WARN Utils: Your hostname, anyes-Latitude-5480 resolves to a loopback address: 127.0.1.1; using 10.51.47.128 instead (on interface wlp2s0)
24/02/23 15:42:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/anyes/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/anyes/.ivy2/cache
The jars for the packages stored in: /home/anyes/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6da74f21-c7b6-448b-b310-bed0dece0f10;1.0
	confs: [default]
	found graphframes#graphframes;0.8.3-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 562ms :: artifacts dl 16ms
	:: modules in use:
	graphframes#graphframes;0.8.3-spark3.5-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

session started, its id is  local-1708699337622


# Lecture des fichiers du dossier BDLE_10K

In [17]:
# Initialisez une session Spark
spark = SparkSession.builder.appName("Proteines_graph").getOrCreate()

parquet_folder = "./local/data/BDLE_10K"

# Récuperer tous les fichiers Parquet compressé avec Snappy
parquet_files = parquet_folder + "/*.snappy.parquet"

#Création du data frame
df = spark.read.format("parquet").option("compression", "snappy").load(parquet_files)

# Affichez les 10 premiére ligne du DataFrame
df.show(10)


+-----------+-----------+-----+
|     seqID1|     seqID2|  sim|
+-----------+-----------+-----+
|117761605:5|152890023:5| 97.8|
|152904885:3|155591878:2| 88.5|
|152887848:4|153682181:0|100.0|
|152937692:5| 80009514:2| 82.3|
|152990923:2|154549183:4| 98.0|
|152867782:1|153171917:1| 83.1|
| 15111981:2|153137370:1|100.0|
|152794195:0| 15280704:2| 96.9|
| 62963742:1| 63783418:5| 98.4|
|152170568:3|153062631:2| 97.6|
+-----------+-----------+-----+
only showing top 10 rows



# Partionnement des arêtes en plusieurs sous ensembles

In [14]:

# Nombre de partitions
num_partitions = 20

# Ajout d'une nouvelle colonne 'partition' basée sur le modulo de seqID1
#F.hash(col('seqID1') Calcul le hash d'une colonne pour partitionner les données.
df_partitioned = df.withColumn('partition', (F.hash(col('seqID1')) % num_partitions + num_partitions) % num_partitions)

# Afficher le DataFrame partitionné
df_partitioned.show()


+-----------+-----------+-----+---------+
|     seqID1|     seqID2|  sim|partition|
+-----------+-----------+-----+---------+
|117761605:5|152890023:5| 97.8|        7|
|152904885:3|155591878:2| 88.5|        0|
|152887848:4|153682181:0|100.0|        0|
|152937692:5| 80009514:2| 82.3|        0|
|152990923:2|154549183:4| 98.0|        0|
|152867782:1|153171917:1| 83.1|       15|
| 15111981:2|153137370:1|100.0|        6|
|152794195:0| 15280704:2| 96.9|        0|
| 62963742:1| 63783418:5| 98.4|       17|
|152170568:3|153062631:2| 97.6|        5|
|152904832:4|154500443:5| 98.6|        0|
|152745429:3| 22988511:2|100.0|        0|
|153161980:3|154256473:0|100.0|        0|
|152903373:4|154426964:5| 80.4|        0|
|146010871:1|158434400:4| 98.7|        2|
|153040013:0|156563807:1| 96.5|        0|
|152933621:4|157634534:2| 96.8|        0|
|153141720:0|155282497:0| 93.2|        0|
|153082115:5|153887336:3| 90.0|        0|
|153025447:3|155660759:1| 97.4|        0|
+-----------+-----------+-----+---

In [18]:
#La fréquence de chaque partition
partition_freq = df_partitioned.groupBy('partition').count()
partition_freq.show()

+---------+-----+
|partition|count|
+---------+-----+
|       14|  883|
|       12|  821|
|       18|  752|
|       13|  835|
|       15|  859|
|        6|  862|
|       17|  819|
|        9|  907|
|       19|  810|
|       16|  888|
|        5|  770|
|        2| 1170|
|       10|  878|
|        4|  741|
|        7|  883|
|        0| 4906|
|        1|  771|
|        8|  763|
|        3|  772|
|       11|  752|
+---------+-----+



In [19]:
# Créer un DataFrame pour chaque partition
dfs_partitioned = []
for partition_id in range(num_partitions):
    df_partition = df_partitioned.filter(col('partition') == partition_id)
    dfs_partitioned.append(df_partition)

# Afficher les DataFrames de chaque partition
for i, df_partition in enumerate(dfs_partitioned):
    print(f"Partition {i}:")
    df_partition.show()

Partition 0:
+-----------+-----------+-----+---------+
|     seqID1|     seqID2|  sim|partition|
+-----------+-----------+-----+---------+
|152904885:3|155591878:2| 88.5|        0|
|152887848:4|153682181:0|100.0|        0|
|152937692:5| 80009514:2| 82.3|        0|
|152990923:2|154549183:4| 98.0|        0|
|152794195:0| 15280704:2| 96.9|        0|
|152904832:4|154500443:5| 98.6|        0|
|152745429:3| 22988511:2|100.0|        0|
|153161980:3|154256473:0|100.0|        0|
|152903373:4|154426964:5| 80.4|        0|
|153040013:0|156563807:1| 96.5|        0|
|152933621:4|157634534:2| 96.8|        0|
|153141720:0|155282497:0| 93.2|        0|
|153082115:5|153887336:3| 90.0|        0|
|153025447:3|155660759:1| 97.4|        0|
|152921945:1|157742939:2| 92.6|        0|
|152901882:2| 79639334:0| 81.7|        0|
|152772567:4|153848527:5| 90.3|        0|
|152897014:5|155771540:2| 84.1|        0|
|152778738:2|153768354:1| 98.2|        0|
|152985004:4|156710084:4| 85.7|        0|
+-----------+--------